In [ ]:
# general packages
import numpy as np
import cv2
import napari
import time
import os
from datetime import date

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage
from aicsimageio.writers import OmeTiffWriter
from aicsimageio.readers.bioformats_reader import BioformatsReader
from aicsimageio.readers.czi_reader import CziReader

# function for core algorithm
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice, image_smoothing_gaussian_3d, edge_preserving_smoothing_3d
from skimage.morphology import remove_small_objects, dilation, erosion, ball     # function for post-processing (size filter)
from skimage.segmentation import watershed, clear_border
from skimage.feature import peak_local_max
from skimage.measure import label
from skimage.io import imsave
from scipy.ndimage import distance_transform_edt, binary_fill_holes

In [ ]:
# load data from .czi format

# User inputs the file path
filePath = "//research.files.med.harvard.edu/sysbio/MEGASON LAB/People/AntoineRuzette/Data/Zebrafish neural tube/20220721_memNG_h2bScarlet_7hpf_40X-01-Airyscan Processing-01.czi"

# Load the image using the AICS importer
img = AICSImage(filePath, reader=CziReader)

# Image information
micronsPerPixel = [img.physical_pixel_sizes.Z, img.physical_pixel_sizes.X , img.physical_pixel_sizes.Y]
print(img.dims)
print(f'Scale of ZXY axis: [Z: {micronsPerPixel[0]}, X: {micronsPerPixel[1]}, Y: {micronsPerPixel[2]}]')

In [ ]:
# view first image - optional

# retrieve specific chunk
raw_img = img.get_image_dask_data('CYX', T=1, Z=0)

# channel axis expands on the dimension specified by its index in 'TCZYX'; if channel_axis = 0, creates a layer for each timestamp       
viewer = napari.view_image(raw_img, contrast_limits=(9000, 23000), channel_axis=0, name=['nuclei', 'membrane'], show = True)

# optional
viewer.scale_bar
viewer.axes.visible = True
viewer.axes.colored = False
viewer.scale_bar.visible = True
viewer.scale_bar.unit = "um"
viewer.scale_bar.ticks = False

In [ ]:
# function selecting a certain slice at a certain timestamp

def slice(img, folder_path, counter, time: int, z_slice: int):
    
    if time > img.dims.T:
        print('Time is out of bound.')
    elif z_slice > img.dims.Z:
        print('Slice is out of bound.')
    else: 
        slice_z = img.get_image_dask_data('CYX', T=time, Z=z_slice)
        viewer = napari.view_image(slice_z, contrast_limits=(9000, 23000), channel_axis=0, show = True)
        slice_name = 'img_' + str(counter) + '.png'
        slice_path = folder_path + '/' + slice_name
        tmp = viewer.screenshot()
        tmp = tmp[135:1020, 25:915]
        imsave(slice_path, tmp)
        viewer.close()

        # TODO
        # resize to 900x900 the images
        # make the image saving faster
        # improve contrast, or color normalization

    return slice

In [ ]:
# pull an user-specified chunk - sclicing for time and z dimensions

##################################################################################################################################
# loaded image
img = img

# time slicing
time1 = 0
time2 = img.dims.T
time_step = 1

# z slicing 
z1 = 0
z2 = img.dims.Z
z_step = 1

# output folder
out = "//research.files.med.harvard.edu/sysbio/MEGASON LAB/People/AntoineRuzette/Data"
folder_name = 'NeuralTube_2D'
folder_path = out + '/' + folder_name + '_' + str(date.today())
log_path = folder_path + '/' + 'log.txt'
###################################################################################################################################         

if ~os.path.isdir(folder_path):
    os.mkdir(folder_path)

i = 0;  
for time in range(time1, time2, time_step):

    sub_folder_path = folder_path + '/' + 'T' + str(time)
    os.mkdir(sub_folder_path)

    for z in range(z1, z2, z_step): 
        slice_z = slice(img, sub_folder_path, i, time, z)
        log_str = str(f'Slice (t{time}; z{z}; counter: {i}) successfully exported!')
        print(log_str)

        # write in log file
        with open(log_path, 'a') as log:
            log.write(log_str + '\n')

        i+=1 